In [2]:
import json
from collections import Counter

In [15]:
json_dir = '../json_data/'
with open('../../webqa/WebQA/data/WebQA_train_val.json','r') as f:
    train_data = json.load(f)
with open('../../webqa/WebQA/data/WebQA_test.json','r') as f:
    test_data = json.load(f)

In [16]:
print(Counter([train_data[k]['split'] for k in train_data]))
print(len(set([train_data[k]['Guid'] for k in train_data])))
print(Counter([train_data[k]['Qcate'] for k in train_data]))

Counter({'train': 36766, 'val': 4966})
41732
Counter({'text': 20267, 'YesNo': 7320, 'Others': 5412, 'choose': 4220, 'number': 2118, 'color': 1830, 'shape': 565})


In [101]:
test_data['d5bbc4440dba11ecb1e81171463288e9']['img_Facts']

[{'image_id': 30033617,
  'title': 'Hall Park Academy Exterior',
  'caption': 'Hall Park Academy Exterior Photograph of the exterior of Hall Park Academy showing the main reception area and part of the school building.',
  'url': 'https://commons.wikimedia.org/wiki/File:Hall_Park_Academy_Exterior.jpg',
  'imgUrl': 'https://upload.wikimedia.org/wikipedia/commons/thumb/3/39/Hall_Park_Academy_Exterior.jpg/800px-Hall_Park_Academy_Exterior.jpg'},
 {'image_id': 30090059,
  'title': 'The Original Playboy Mansion',
  'caption': 'The Original Playboy Mansion 1340 N State Pkwy Chicago, IL 60610',
  'url': 'https://commons.wikimedia.org/wiki/File:The_Original_Playboy_Mansion.jpg',
  'imgUrl': 'https://upload.wikimedia.org/wikipedia/commons/thumb/1/14/The_Original_Playboy_Mansion.jpg/800px-The_Original_Playboy_Mansion.jpg'},
 {'image_id': 30101667,
  'title': 'Moorside High School, Cellarhead - geograph.org.uk - 340365',
  'caption': 'Moorside High School, Cellarhead - geograph.org.uk - 340365 Moo

In [21]:
train_data['d5bbc6d80dba11ecb1e81171463288e9']['img_posFacts']

[{'image_id': 30321533,
  'title': 'National Museum of the American Indian in Washington, D.C',
  'caption': 'National Museum of the American Indian in Washington, D.C',
  'url': 'https://commons.wikimedia.org/wiki/File:National_Museum_of_the_American_Indian_in_Washington%2C_D.C.jpg',
  'imgUrl': 'https://upload.wikimedia.org/wikipedia/commons/thumb/b/ba/National_Museum_of_the_American_Indian_in_Washington%2C_D.C.jpg/800px-National_Museum_of_the_American_Indian_in_Washington%2C_D.C.jpg'},
 {'image_id': 30278153,
  'title': 'Xanadu-House-in-Kissimmee-Florida-1985',
  'caption': 'Xanadu-House-in-Kissimmee-Florida-1985 A photo of the Xanadu House that was located in Kissimmee, Florida, showing the exterior of the house.',
  'url': 'https://commons.wikimedia.org/wiki/File:Xanadu-House-in-Kissimmee-Florida-1985.jpg',
  'imgUrl': 'https://upload.wikimedia.org/wikipedia/commons/8/8f/Xanadu-House-in-Kissimmee-Florida-1985.jpg'}]

In [34]:
train_data['d5bbc6d80dba11ecb1e81171463288e9']

{'Q': '"Are both the National Museum of the American Indian in Washington, D.C. and the Xanadu House in Kissimmee, Florida the same color?"',
 'A': ['"Yes, both the National Museum of the American Indian in Washington, D.C. and the Xanadu House in Kissimmee, Florida are beige."'],
 'topic': 'strange architecture',
 'split': 'train',
 'Qcate': 'YesNo',
 'Guid': 'd5bbc6d80dba11ecb1e81171463288e9',
 'img_posFacts': [{'image_id': 30321533,
   'title': 'National Museum of the American Indian in Washington, D.C',
   'caption': 'National Museum of the American Indian in Washington, D.C',
   'url': 'https://commons.wikimedia.org/wiki/File:National_Museum_of_the_American_Indian_in_Washington%2C_D.C.jpg',
   'imgUrl': 'https://upload.wikimedia.org/wikipedia/commons/thumb/b/ba/National_Museum_of_the_American_Indian_in_Washington%2C_D.C.jpg/800px-National_Museum_of_the_American_Indian_in_Washington%2C_D.C.jpg'},
  {'image_id': 30278153,
   'title': 'Xanadu-House-in-Kissimmee-Florida-1985',
   'cap

In [38]:
# load mapping condition
mapping = '../../newqa/WebQA_Baseline/misc/image_id_map_0328.pkl'
import pickle
with open(mapping, 'rb') as f:
    img2featid = pickle.load(f)

In [51]:
from tqdm import tqdm
import os
bad_pos_sources = []
bad_neg_sources = []
total_count = 0
health_webqa = {}
for k in tqdm(train_data):

    posCount = negCount = 0
    for imgdir in train_data[k]['img_posFacts']:
        imgpath = ('/ocean/projects/cis210027p/shared/corpora/webqa/%s/%s.pkl'%(train_data[k]['split'], img2featid[imgdir['image_id']])).replace('val','dev')
        if os.path.exists(imgpath):
            posCount += 1
    for imgdir in train_data[k]['img_negFacts']:
        imgpath = ('/ocean/projects/cis210027p/shared/corpora/webqa/%s/%s.pkl'%(train_data[k]['split'], img2featid[imgdir['image_id']])).replace('val','dev')
        if os.path.exists(imgpath):
            negCount += 1
    # if posCount < 2:
    #     bad_pos_sources.append(k)
    # if not negCount:
    #     bad_neg_sources.append(k)
    if posCount > 0 and negCount > 0:
        total_count += 1
        health_webqa[k] = train_data[k]

100%|██████████| 41732/41732 [00:06<00:00, 6032.74it/s]


In [52]:
total_count

15920

In [53]:
len(health_webqa)

15920

In [47]:
len(bad_neg_sources)

1946

In [48]:
len(bad_pos_sources)

37207

In [32]:
len(bad_neg_sources)

41732

### Todos
- [x] category statistics
- [x] making up subset

#### Category Statistics

In [11]:
len(train_data)

41732

In [71]:
qcate2count = dict(Counter([health_webqa[k]['Qcate'] for k in health_webqa]))
# del qcate2count['text']

In [72]:
qcate2count

{'YesNo': 5775,
 'Others': 3798,
 'color': 1280,
 'choose': 3311,
 'shape': 390,
 'number': 1366}

In [56]:
counts = qcate2count.values()
ratios = [count/sum(counts) for count in counts]
qcate2ratio = dict(zip(list(qcate2count.keys()), ratios))

In [57]:

qcate2ratio

{'YesNo': 0.36275125628140703,
 'Others': 0.2385678391959799,
 'color': 0.08040201005025126,
 'choose': 0.20797738693467338,
 'shape': 0.024497487437185928,
 'number': 0.08580402010050252}

In [24]:
subsetSize = 10000
qcate2hc = {qcate:int(subsetSize*ratio) for (qcate,ratio) in qcate2ratio.items()}

In [58]:
qcate2hc = {'YesNo':2500, 'Others':2000, 'number':1500, 'color':1500, 'choose':2000, 'shape':1000}

In [59]:
subWebqa = {}
import collections
from tqdm import tqdm
qcate2items = collections.defaultdict(list)
for k in tqdm(health_webqa):
    if health_webqa[k]['Qcate'] in qcate2hc:
        qcate2items[health_webqa[k]['Qcate']].append((k, health_webqa[k]))
from random import shuffle


100%|██████████| 15920/15920 [00:00<00:00, 932510.12it/s]


In [60]:
qcate2hc

{'YesNo': 2500,
 'Others': 2000,
 'number': 1500,
 'color': 1500,
 'choose': 2000,
 'shape': 1000}

In [64]:
import random
random.seed(11777)
for qcate in qcate2items:
    currPoolChoices = random.sample(qcate2items[qcate], min(len(qcate2items[qcate]), qcate2hc[qcate]))
    print(len(currPoolChoices))
    for k, v in currPoolChoices:
        subWebqa[k] = v

2500
2000
1280
2000
390
1366


In [62]:
len(subWebqa)

9536

In [63]:
print(Counter([subWebqa[k]['split'] for k in subWebqa]))
print(len(set([subWebqa[k]['Guid'] for k in subWebqa])))
print(Counter([subWebqa[k]['Qcate'] for k in subWebqa]))

Counter({'train': 8231, 'val': 1305})
9536
Counter({'YesNo': 2500, 'Others': 2000, 'choose': 2000, 'number': 1366, 'color': 1280, 'shape': 390})


In [49]:
len(random.choices(qcate2items['YesNo'], k = 2000))

2000

#### Make test_data subset

In [104]:
test_qcate2count = dict(Counter([test_data[k]['Qcate'] for k in test_data]))
# del test_qcate2count['text']

KeyError: 'Qcate'

In [103]:
len(test_data)

7540

In [105]:
testSubWebqa = {k:v for k,v in list(random.sample(healthy_test_Webqa.items() ,1543))}

/tmp/ipykernel_92320/2648213638.py:1: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  testSubWebqa = {k:v for k,v in list(random.sample(healthy_test_Webqa.items() ,1543))}


In [106]:
len(testSubWebqa)

1543

In [71]:
# testSubWebqa['d5c9dd040dba11ecb1e81171463288e9']

In [72]:
# test_data['d5bbc4440dba11ecb1e81171463288e9']

In [74]:
train_subset = {k:v for k,v in subWebqa.items() if subWebqa[k]['split'] == 'train'}
valid_subset = {k:v for k,v in subWebqa.items() if subWebqa[k]['split'] == 'val'}

In [75]:
len(train_subset)

8231

In [76]:
len(valid_subset)

1305

In [108]:
len(testSubWebqa)

1543

In [98]:
healthy_test_Webqa = {}

In [99]:
# choose test json
for k in tqdm(test_data):
    imgCount = 0
    for imgdir in test_data[k]['img_Facts']:
        imgpath = ('/ocean/projects/cis210027p/shared/corpora/webqa/%s/%s.pkl'%(test_data[k]['split'], img2featid[imgdir['image_id']]))
        if os.path.exists(imgpath):
            imgCount += 1
    if imgCount > 10:
        healthy_test_Webqa[k] = test_data[k]

100%|██████████| 7540/7540 [00:01<00:00, 6744.13it/s]


In [107]:
len(healthy_test_Webqa)

4710

In [77]:
print(Counter([train_subset[k]['split'] for k in train_subset]))
print(len(set([train_subset[k]['Guid'] for k in train_subset])))
print(Counter([train_subset[k]['Qcate'] for k in train_subset]))

Counter({'train': 8231})
8231
Counter({'YesNo': 2179, 'choose': 1721, 'Others': 1705, 'number': 1163, 'color': 1128, 'shape': 335})


In [78]:
print(Counter([valid_subset[k]['split'] for k in valid_subset]))
print(len(set([valid_subset[k]['Guid'] for k in valid_subset])))
print(Counter([valid_subset[k]['Qcate'] for k in valid_subset]))

Counter({'val': 1305})
1305
Counter({'YesNo': 321, 'Others': 295, 'choose': 279, 'number': 203, 'color': 152, 'shape': 55})


In [109]:
with open('../subWebqa/data/train_subWebqa.json','w')as f:
    json.dump(train_subset, f)
with open('../subWebqa/data/val_subWebqa.json','w') as f:
    json.dump(valid_subset, f)
with open('../subWebqa/data/test_subWebqa.json','w') as f:
    json.dump(testSubWebqa, f)